# QC for location, trajecotry and CUD papers

In [1]:
from utils_project import *
from functools import reduce
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.formula.api as smf
from fmriqc import * 
from socialspace import * 

Detected user: /Users/matty_gee
Base directory: /Users/matty_gee/Desktop/projects/trajectory
Included n=50
Found 39 mask nifties


# Get & organize data

In [13]:
df  = pd.read_excel(f'{base_dir}/All-data_summary_n105.xlsx')
print(f'n={df.shape[0]}')

n=105


In [14]:
roles = character_roles
for role in character_roles:
    df = df.rename(columns={f'dots_{role}_affil': f'{role}_dots_affil',
                             f'dots_{role}_power': f'{role}_dots_power'})
    
#--------------------------------------------------------------------------------------------------
# neutral character values
#--------------------------------------------------------------------------------------------------

print('Adding neutral character values')

df[['affil_coord_neutral', 'power_coord_neutral', 'affil_mean_neutral', 'power_mean_neutral']] = 0, 0, 0, 0
df[['neu_2d_dist_neutral', 'neu_2d_dist_mean_neutral', 'pov_2d_dist_neutral', 'pov_2d_dist_mean_neutral']] = 0, 0, 6, 6
df[['pov_2d_angle_neutral', 'neu_2d_angle_neutral']] = np.deg2rad(90), np.deg2rad(0)

# adjust the dots & task variables using neutral dots location
for dim in ['affil', 'power']:

    # neutral dots coordinates
    adjust = df[f'neutral_dots_{dim}'].values[:,np.newaxis]

    # dots
    cols = [f'{r}_dots_{dim}' for r in character_roles]
    df[[f'{c}_adj' for c in cols]] = df[cols].values - adjust
    df[f'dots_{dim}_mean_adj'] = np.mean(df[[f'{c}_adj' for c in cols]], axis=1)

    # task
    cols = [f'{dim}_mean_{r}' for r in character_roles]
    df[[f'{c}_adj' for c in cols]] = df[cols].values - adjust
    df[f'{dim}_mean_mean_adj'] = np.mean(df[[f'{c}_adj' for c in cols]], axis=1)

#--------------------------------------------------------------------------------------------------
# dots: Validation only
#---------------------------------------------------------------------------------------------------

dots_df = pd.DataFrame(columns=['sub_id'])
for r, row in df.iterrows():

    print(f"Adding dots variables: {r+1} / {len(df)}", end='\r')
    dots_df.loc[r, 'sub_id'] = row['sub_id']

    for adj in ['_adj', '']:

        #--------------------------------------------------------------------------------------------------
        # behavior
        #--------------------------------------------------------------------------------------------------

        beh_cols = flatten_lists([[f'affil_mean_{r}{adj}', f'power_mean_{r}{adj}'] for r in roles])
        beh_xy = row[beh_cols].values.astype(float).reshape(-1, 2) # scaled to [-1, 1]
        beh_pov_dists = np.array([norm(xy-[1,0]) for xy in beh_xy]).astype(float).reshape(-1, 1)
        beh_neu_dists = np.array([norm(xy) for xy in beh_xy]).astype(float).reshape(-1, 1)

        #--------------------------------------------------------------------------------------------------
        # dots
        #--------------------------------------------------------------------------------------------------

        dots_cols = flatten_lists([[f'{r}_dots_affil{adj}', f'{r}_dots_power{adj}'] for r in roles])
        dots_xy = row[dots_cols].values.astype(float).reshape(-1, 2)
        if np.isnan(dots_xy).any(): continue

        dots_df.loc[r, [f'dots_affil_mean{adj}', f'dots_power_mean{adj}']] = np.mean(dots_xy, axis=0)
        dots_df.loc[r, [f'dots_surface_area{adj}', f'dots_area{adj}']] = ComputeBehavior2.calc_shape_size(dots_xy)
        dots_df.loc[r, f'dots_avg_pw_dist{adj}'] = np.mean(get_rdv(dots_xy), axis=0)

        # area of convex hull
        try: 
            area = scipy.spatial.ConvexHull(dots_xy).volume # volume = area when in 2D
        except:
            area = np.nan
        dots_df.loc[r, 'area'] = area

        # distance from self
        dots_pov_dists = np.array([norm(xy-[1, 0]) for xy in dots_xy]).astype(float).reshape(-1, 1) # euclidean distances from reference to xy
        dots_df.loc[r, [f'dots_pov_dist_{r}{adj}' for r in roles]] = dots_pov_dists.flatten()
        dots_df.loc[r, f'dots_pov_dist_mean{adj}'] = np.mean(dots_pov_dists, axis=0)

        # distance from origin
        dots_neu_dists = np.array([norm(xy) for xy in dots_xy]).astype(float).reshape(-1, 1) # euclidean distances from origin to xy
        dots_df.loc[r, [f'dots_neu_dist_{r}{adj}' for r in roles]] = dots_neu_dists.flatten()
        dots_df.loc[r, f'dots_neu_dist_mean{adj}'] = np.mean(dots_neu_dists, axis=0)

        #--------------------------------------------------------------------------------------------------
        # behavior vs. dots
        #--------------------------------------------------------------------------------------------------

        # single dimension distances
        xy_diff = beh_xy - dots_xy
        dots_df.loc[r, [f'beh_dots_affil_diff_{r}{adj}' for r in roles]] = xy_diff[:,0]
        dots_df.loc[r, f'beh_dots_affil_diff_mean{adj}'] = np.mean(xy_diff[:,0])
        dots_df.loc[r, f'beh_dots_affil_diff_std{adj}'] = np.std(xy_diff[:,0])
        dots_df.loc[r, [f'beh_dots_power_diff_{r}{adj}' for r in roles]] = xy_diff[:,1]
        dots_df.loc[r, f'beh_dots_power_diff_mean{adj}'] = np.mean(xy_diff[:,1])
        dots_df.loc[r, f'beh_dots_power_diff_std{adj}'] = np.std(xy_diff[:,1])

        # distances between behavior and dots
        beh_dots_dists     = norm(beh_xy - dots_xy, axis=1)
        beh_dots_pov_dists = norm(beh_pov_dists - dots_pov_dists, axis=1)
        dots_df.loc[r, [f'beh_dots_pov_dist_diff_{r}{adj}' for r in roles]] = beh_dots_pov_dists.flatten()
        dots_df.loc[r, [f'beh_dots_dist_{r}{adj}' for r in roles]] = beh_dots_dists.flatten()
        dots_df.loc[r, f'beh_dots_dist_mean{adj}'] = np.mean(beh_dots_dists, axis=0)
        dots_df.loc[r, f'beh_dots_dist_std{adj}'] = np.std(beh_dots_dists, axis=0)

        # permutation analysis: are permuted locations further away from the behavior than the real dots locations?
        dots_shuffs = [np.mean(norm(beh_xy - np.random.permutation(dots_xy), axis=1)) for _ in range(1000)]
        dots_df.loc[r, f'beh_dots_dist_shuff_mean{adj}'] = np.mean(dots_shuffs)
        dots_df.loc[r, f'beh_dots_dist_shuff_std{adj}']  = np.std(dots_shuffs)

        # RSAs for behavior and dots
        dots_df.loc[r, f'beh_dots_pw_dist_tau{adj}']  = kendalltau(get_rdv(dots_xy), get_rdv(beh_xy))[0]
        dots_df.loc[r, f'beh_dots_pov_dist_tau{adj}'] = kendalltau(get_rdv(dots_pov_dists), get_rdv(beh_pov_dists))[0]
        dots_df.loc[r, f'beh_dots_neu_dist_tau{adj}'] = kendalltau(get_rdv(dots_neu_dists), get_rdv(beh_neu_dists))[0]

        # procrustes transformation 
        # rotationm reflection, scaling & translation: disparity = sum of the squares of the point-wise euclidean distances
        dots_df.loc[r, 'beh_dots_procrustes_disp'] = scipy.spatial.procrustes(dots_xy, beh_xy)[2] # order doesnt matter

        # orthogonal: rotation & reflection (no scaling or translation)
        dots_df.loc[r, 'beh_dots_orth_procrustes_scale'] = scipy.linalg.orthogonal_procrustes(beh_xy, dots_xy)[1]  # order doesnt matter

        # least squares: task @ transform = dots
        try: 
            _, res, _ , _ = np.linalg.lstsq(beh_xy, dots_xy, rcond=None)
            dots_df.loc[r, ['beh_dots_lstsqres_affil', 'beh_dots_lstsqres_power']] = res[0], res[1]
        except:
            dots_df.loc[r, ['beh_dots_lstsqres_affil', 'beh_dots_lstsqres_power']] = np.nan, np.nan
        

        #--------------------------------------------------------------------------------------------------
        # memory rsa
        #--------------------------------------------------------------------------------------------------

        dists_diff_rdv = get_rdv(beh_dots_dists.reshape(-1,1)) # distance between dots and task as an rdv
        memory_rdv = get_rdv(row[[f'memory_{r}' for r in roles]].values.reshape(-1,1))
        dots_df.loc[r, f'beh_dots_dist_diff_memory_tau{adj}'] = kendalltau(dists_diff_rdv, memory_rdv)[0]
    
dots_df.iloc[:, 1:] = dots_df.iloc[:, 1:].astype(float)
df = df.merge(dots_df, on='sub_id', how='outer')
print(f'df shape = {df.shape}')

Adding neutral character values
df shape = (105, 2824) 105 / 105


In [17]:
# # initial data
# init_df  = pd.read_excel(f'{data_dir}/Initial/Summary/All-data_summary_n21.xlsx')
# print(f'Initial n={init_df.shape[0]}')

# # merge cud_Df and init_df 
# df = pd.concat([cud_df, init_df])
# df.drop(columns=['snt_ver'], inplace=True)

# validation sample LSAS scores
fear      = np.sum(df[['lsas_social_interaction_fear_subscale','lsas_performance_fear_subscale']], axis=1).values.astype(int)
avoidance = np.sum(df[['lsas_social_interaction_avoidance_subscale', 'lsas_performance_avoidance_subscale']], axis=1).values.astype(int)
df['lsas_av_score2'] = avoidance + df['lsas_av_score'].fillna(0).values.astype(int)
df['lsas_fear_score2'] = fear + df['lsas_fear_score'].fillna(0).values.astype(int)

# sex variable
df['sex'].replace({'male': 'M', 'female': 'F'}, inplace=True)

print(f'All n={df.shape[0]}')
display(df.head(8))

df.to_excel(f'{data_dir}/All-data_summary_n{len(df)}.xlsx', index=False)

All n=105


,sub_id,sample,dx,map_incl,map excl reason,cud_incl,cud excl reason,fd_mean,fd_max,fd_decision_mean,...,beh_dots_dist_boss,beh_dots_dist_neutral,beh_dots_dist_mean,beh_dots_dist_std,beh_dots_dist_shuff_mean,beh_dots_dist_shuff_std,beh_dots_pw_dist_tau,beh_dots_pov_dist_tau,beh_dots_neu_dist_tau,beh_dots_dist_diff_memory_tau
0,1,Initial,HC,1.0,NaN,NaN,NaN,0.073544,0.257525,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Initial,HC,1.0,NaN,NaN,NaN,0.037199,0.368889,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Initial,HC,1.0,NaN,NaN,NaN,0.081574,0.802291,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Initial,HC,NaN,missing data - data corruption in archiving,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Initial,HC,1.0,NaN,NaN,NaN,0.097804,2.037662,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Initial,HC,1.0,NaN,NaN,NaN,0.061781,0.335238,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Initial,HC,1.0,NaN,NaN,NaN,0.094421,0.937448,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Initial,HC,1.0,NaN,NaN,NaN,0.082026,0.304559,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Memory

In [ ]:
sns.histplot(data['memory_mean_main'])
plt.show()

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multitest import multipletests

def run_anova(data, correction_method='bonferroni'):

    # Check if data is a numpy array
    if isinstance(data, np.ndarray):
        df = pd.DataFrame(data)
        df.columns = [f'Event{i+1}' for i in range(data.shape[1])]
    elif isinstance(data, pd.DataFrame):
        df = data
    else:
        raise TypeError("Input data should be either a pandas DataFrame or a numpy array.")
        
    columns = df.columns.tolist()

    # Reshape the data
    df_melt = pd.melt(df.reset_index(), id_vars=['index'], value_vars=columns)
    df_melt.columns = ['index', 'Event', 'Score']

    assumptions = True

    # normality across groups
    for event in df_melt['Event'].unique():
        norm_p = scipy.stats.shapiro(df_melt[df_melt['Event'] == event]['Score'])[1]
        if norm_p < 0.05:
            assumptions = False

    # homogeneity of variance
    if assumptions: 
        # bartlett test
        homo_p = scipy.stats.bartlett(*[df_melt[df_melt['Event'] == event]['Score'] for event in df_melt['Event'].unique()])[1]
    else:
        homo_p = scipy.stats.levene(*[df_melt[df_melt['Event'] == event]['Score'] for event in df_melt['Event'].unique()])[1]
    if homo_p < 0.05:
        assumptions = False

    if assumptions: 
        # Run one-way ANOVA
        anova_table = sm.stats.anova_lm(ols('Score ~ C(Event)', data=df_melt).fit(), typ=2)
        F, anova_p = anova_table.values[0, 2:3]
        print(f'One-way ANOVA: F = {F:.02f}, p = {anova_p:.04f}')
    else: 
        # Run Kruskal-Wallis test
        H, anova_p = scipy.stats.kruskal(*[df[col].values for col in columns])
        print(f'Kruskal-Wallis H-test: H = {H:.02f}, p = {anova_p:.04f}')

    # Run post-hoc tests
    ps = []
    for combo in list(itertools.combinations(columns, 2)):
        if assumptions: 
            t, p = scipy.stats.ttest_rel(df[combo[0]], df[combo[1]])
        else:
            w, p = scipy.stats.wilcoxon(df[combo[0]], df[combo[1]])
        ps.append(p)
    reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(ps, alpha=0.05, method=correction_method)
    df_pvals = pd.DataFrame()
    df_pvals['comparison'] = [f'{combo[0]} vs {combo[1]}' for combo in list(itertools.combinations(columns, 2))]
    df_pvals['p-value'] = pvals_corrected
    df_pvals['reject_h0'] = reject
    return df_pvals

# Run ANOVA for memory
data_ = sample_dict['Validation']['data']
data_ = data_[data_['memory_mean'] > 0.2]
print(f'n={len(data_)}')
run_anova(data_[memory_cols])

# Summarize motion (FD)

In [3]:
from info import task

def summarize_rp(rp_txt, sample):

    sub_id = rp_txt.split('/')[-1].split('_')[0]

    # dataframe for summary
    motion_measures = ['fd']
    col_list = []
    for mm in motion_measures:
        col_list.extend([f'{mm}_mean', f'{mm}_max', f'{mm}_decision_mean', f'{mm}_decision_max',
                         f'{mm}_nondecision_mean', f'{mm}_nondecision_max'])
    out_df = pd.DataFrame(columns=['sub_id'] + col_list)
    
    # define tr to get decision periods
    tr, vox = (1.0, 2.1) if sample == 'Validation' else (2.0, 3.0)
    task_df = get_task_trs(tr=tr)
    decision_trs = (task_df['trial_type'] == 'Decision').values
    nondecision_trs = (task_df['trial_type'] != 'Decision').values

    print(f'Processing {sub_id}', end='\r')

    #-----------------------------------------------
    # summarize motion parameters
    #-----------------------------------------------       
        
    rp = pd.read_csv(rp_txt, header=None, delim_whitespace=True)
    if rp.shape[1] != 6:
        rp = pd.read_csv(rp_txt, header=None)

    rp.columns = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']
    rp.insert(0, 'fd', calc_fd(rp))

    out_df.loc[0, 'sub_id'] = sub_id
    for measure in motion_measures: 
        
        out_df.loc[0, f'{measure}_mean'] = np.nanmean(rp[measure], 0)
        out_df.loc[0, f'{measure}_max']  = np.max(rp[measure], 0)

        try: 
            out_df.loc[0, f'{measure}_decision_mean'] = np.nanmean(rp[decision_trs][measure], 0)
            out_df.loc[0, f'{measure}_nondecision_mean'] = np.nanmean(rp[nondecision_trs][measure], 0)

            out_df.loc[0, f'{measure}_decision_max']  = np.max(rp[decision_trs][measure], 0)
            out_df.loc[0, f'{measure}_nondecision_max']  = np.max(rp[nondecision_trs][measure], 0)

            out_df.loc[0, f'{measure}_decision_greater-halfvox']  = np.sum(rp[decision_trs][measure] > (vox/2))
            out_df.loc[0, f'{measure}_nondecision_greater-halfvox']  = np.sum(rp[nondecision_trs][measure] > (vox/2))
            out_df.loc[0, f'{measure}_decision_greater-vox']  = np.sum(rp[decision_trs][measure] > vox)
            out_df.loc[0, f'{measure}_nondecision_greater-vox']  = np.sum(rp[nondecision_trs][measure] > vox)
        except:
            print('Error w/ decision/nondecision motion summary')

    return out_df

def get_task_trs(tr=1.0):

    # double & triple check timing....
    # new data: ... TRs, older data: ... TRs
    
    if tr == 1.0:
        onset, offset = 'cogent_onset', 'cogent_offset'
    elif tr == 2.0:
        onset, offset = 'cogent_onset_2015', 'cogent_offset_2015'

    other_rows = ['trial_type', 'slide_num', 'scene_num', 'dimension', 'char_role_num', 'char_decision_num']
    out = []
    for _, row in task.iterrows():
        on, off = np.round(row[onset]), np.round(row[offset])
        sec_ix = np.arange(on, off, tr)[:, np.newaxis] # range of indices
        other  = np.vstack([np.repeat(r, len(sec_ix)) for r in row[other_rows]]).T
        out.append(np.hstack([sec_ix, other]))

    out_df = pd.DataFrame(np.vstack(out), columns=['onset(s)'] + other_rows)
    out_df['onset(s)'] = out_df['onset(s)'].astype(float).astype(int)
    out_df.insert(0, 'TR', out_df.index + 1)

    # add rows if needed
    total_secs = out_df['onset(s)'].values[-1] # should be 1570
    missing_secs = 1570 - total_secs

    # add rows to tr_df
    extra_trs_df = pd.DataFrame(np.arange(total_secs+1, 1570), columns=['onset(s)'])
    extra_trs_df['TR'] = np.arange(total_secs+1, 1570) / tr
    extra_trs_df[other_rows] = np.nan

    out_df = pd.concat([out_df, extra_trs_df], axis=0)
    return out_df.reset_index(drop=True)


In [ ]:
rp_txts = glob.glob(f'{base_dir}/QC/*rp.txt')
motion_dfs = []
for rp_txt in rp_txts: 
    sub_id = rp_txt.split('/')[-1].split('_')[0]
    sample = 'Validation' if len(sub_id) > 2 else 'Initial'
    try: 
        motion_df = summarize_rp(rp_txt, sample)
        # motion_df.to_excel(f'{base_dir}/QC/{sub_id}_motion.xlsx', index=False)
        motion_dfs.append(motion_df)
    except:
        print(f'Error with {sub_id}')

# summarize
motion_df = pd.concat(motion_dfs).sort_values('sub_id').reset_index(drop=True)
print(f'Motion n={len(motion_df)}')
motion_df.to_excel(f'{data_dir}/motion_n{len(motion_df)}.xlsx', index=False)
motion_df['sub_id'] = motion_df['sub_id'].astype(int)

# Merge files

In [5]:
motion_df = pd.read_excel(glob.glob(f'{data_dir}/motion_n*.xlsx')[0])
incl_df = pd.read_excel(f'{data_dir}/participants_info_n105.xlsx')
df = merge_dfs([incl_df, motion_df, df], how='outer')
df.to_excel(f'{data_dir}/All-data_summary_n{len(df)}.xlsx', index=False)

# SNR

In [ ]:
#---------------------------------------------------------------
# add in SNR
#---------------------------------------------------------------

if 'L_HPC_thr25_tsnr' not in df.columns:
    snr_dfs = glob.glob(f'{base_dir}/QC/*snr*summary.xlsx')
    hpc_snrs = []
    for snr_df in snr_dfs: 
        sub_id = snr_df.split('/')[-1].split('_')[0]
        if len(sub_id) > 2:
            snr_df = pd.read_excel(snr_df)
            hpc_snr = snr_df[snr_df['roi'].isin(['L_HPC_thr25', 'R_HPC_thr25'])]
            hpc_snr = hpc_snr.pivot(index='sub_id', columns=['roi','snr_type'], values='snr')
            hpc_snr.columns = ['_'.join(col).strip() for col in hpc_snr.columns.values]
            hpc_snr = hpc_snr.reset_index()
            hpc_snrs.append(hpc_snr)
    snr_df = pd.concat(hpc_snrs)
    snr_df = snr_df.sort_values('sub_id').reset_index(drop=True)
    print_df(snr_df)
    snr_df.to_excel(f'{cud_dir}/HPC_SNR_n{len(snr_df)}.xlsx', index=False)
    df = df.merge(snr_df, on='sub_id', how='outer')

    # save df 
    df.to_excel(f'{cud_dir}/All-data_summary_n{len(df)}.xlsx', index=False)


# Summarize QC

In [ ]:
def plot_all_qc(df):

    figs = []

    #---------------------------------------------------------------
    # task QC
    #---------------------------------------------------------------

    # distributions by group
    comps = ['ctq_score', 'moca_total', 'memory_mean', 'missed_trials', 'reaction_time_mean']
    fig, axs = plt.subplots(1, len(comps), figsize=(3.3 * len(comps), 3))
    for i, col in enumerate(comps):
        sns.histplot(x=col, data=df[df['dx'] == 'CD'], ax=axs[i], 
                    color='red', alpha=0.5, bins=10)
        sns.histplot(x=col, data=df[df['dx'] == 'HC'], ax=axs[i], 
                    color='blue', alpha=0.5, bins=10)
        df_ = df[~df[col].isna()]
        t, p = scipy.stats.ttest_ind(df_[df_['dx'] == 'CD'][col], df_[df_['dx'] == 'HC'][col])
        axs[i].set_title(f'{col} \nDX t-test p={p:.2f}')
    figs.append(fig)
    plt.tight_layout()

    # pairwise correlations by group
    combos = list(itertools.combinations(comps, 2))
    n_rows = int(np.ceil(len(combos) / 3))
    fig, axs = plt.subplots(n_rows, 3, figsize=(10, 3 * n_rows))
    for i, (x, y) in enumerate(combos):
        ax = axs[int(i / 3), i % 3]
        df_ = df[~df[x].isna() & ~df[y].isna()]
        sns.regplot(x=x, y=y, data=df_[df_['dx'] == 'HC'], ax=ax, 
                    color='blue', scatter_kws={'alpha': 0.5})
        sns.regplot(x=x, y=y, data=df_[df_['dx'] == 'CD'], ax=ax, 
                    color='red', scatter_kws={'alpha': 0.5})

        # OLS interaction effect of dx with x
        model = smf.ols(f'{y} ~ {x} + dx + {x}:dx', data=df_).fit()
        p = model.pvalues[f'{x}:dx[T.HC]']
        ax.set_title(f'{y} ~ {x} * dx\np={p:.2f}', fontsize=12)
        
        ax.set_xlabel(x)
        ax.set_ylabel(y)
    figs.append(fig)
    plt.tight_layout()

    # 3d plot
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(30, 30)
    for dx, color in {'CD': 'red', 'HC': 'blue'}.items():
        df_ = df[df['dx'] == dx]
        ax.scatter(df_['reaction_time_mean'], df_['memory_mean'], df_['missed_trials'], 
                color=color, s=100, alpha=0.5, label=dx)
    for i, txt in enumerate(df['sub_id']):

        if (df['missed_trials'][i] > 5) or \
            (df['memory_mean'][i] < 0.33) or \
            (df['reaction_time_mean'][i] < 2.5):
            ax.text(df['reaction_time_mean'][i], df['memory_mean'][i], df['missed_trials'][i], 
                    txt, fontsize=8)
            
    ax.set_xlabel('Reaction time')
    ax.set_ylabel('Memory')
    ax.set_zlabel('Missed trials')
    figs.append(fig)
    plt.show()

    #---------------------------------------------------------------
    # fMRI QC
    #---------------------------------------------------------------

    # tsnr and ssnr in HPC
    fig, axs = plt.subplots(1, 4, figsize=(12, 3))
    cols = ['L_HPC_thr25_tsnr', 'L_HPC_thr25_ssnr', 'R_HPC_thr25_tsnr', 'R_HPC_thr25_ssnr']
    for i, col in enumerate(cols):
        sns.histplot(x=col, data=df[df['dx'] == 'CD'], ax=axs[i], color='red', alpha=0.5, bins=10)
        sns.histplot(x=col, data=df[df['dx'] == 'HC'], ax=axs[i], color='blue', alpha=0.5, bins=10)
        df_ = df[~df[col].isna()]
        t, p = scipy.stats.ttest_ind(df_[df_['dx'] == 'CD'][col], df_[df_['dx'] == 'HC'][col])
        axs[i].set_title(f'{col} \nDX t-test p={p:.2f}')
    figs.append(fig)
    plt.tight_layout()

    # motion
    cols = ['fd_max', 'fd_decision_max', 'fd_nondecision_max', 'fd_decision_greater-vox']
    fig, axs = plt.subplots(1, len(cols), figsize=(len(cols) * 3, 3))
    for i, col in enumerate(cols):
        df_ = df[~df[col].isna()]
        sns.histplot(df_[col][df_['dx'] == 'HC'], ax=axs[i], 
                     bins=10, alpha=0.5, color='blue', label='HC')
        sns.histplot(df_[col][df_['dx'] == 'CD'], ax=axs[i], 
                     bins=10, alpha=0.5, color='red', label='CD')
        t, p = scipy.stats.ttest_ind(df_[col][df['dx'] == 'HC'], df_[col][df['dx'] == 'CD'])
        axs[i].axvline(x=2.1, color='black', linewidth=3)
        axs[i].set_title(f'{col} \nDX t-test p={p:.2f}', fontsize=12)  
    plt.tight_layout()
    figs.append(fig)
    plt.show()

    save_figures_pdf(figs, f'{cud_dir}/../../QC/QC_plots_n{len(df)}.pdf')

plot_all_qc(df)

In [ ]:
memory_incl = df['memory_mean'] > 0.2
# rt_incl = df['reaction_time_mean'] > 2
missed_incl = df['missed_trials'] < 15
fd_incl = df['fd_decision_greater-vox'] < 5

df_ = df[memory_incl & missed_incl & fd_incl]
df_.reset_index(inplace=True, drop=True)

plot_all_qc(df_)